In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from IPython import get_ipython
get_ipython().run_line_magic('matplotlib','inline')
import copy
import sys,os
import matplotlib.pyplot as plt
from IPython import display
from collections import OrderedDict
import time
#from torch
import torch
from torch import nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler

#from torchvision
import torchvision
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
#from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.utils.data import TensorDataset

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
use_gpu = torch.cuda.is_available()
print("usando gpu:", use_gpu)

In [ ]:
import pandas as pd
sample_submission = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")
train = pd.read_csv("../input/digit-recognizer/train.csv")

In [ ]:
train.head() 

In [ ]:
train.info()

In [ ]:
print('Train shape:', train.shape)
print('Test shape:', test.shape)

In [ ]:
sys.path

In [ ]:
class MEUMNIST(Dataset):
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        image = self.data.iloc[index, 1:].values.astype(np.uint8).reshape((28, 28, 1))
        label = self.data.iloc[index, 0]
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

In [ ]:
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

datasets = {
        'train': MEUMNIST('../input/digit-recognizer/train.csv', transform=data_transform),
        'test':  MEUMNIST('../input/digit-recognizer/test.csv', transform=data_transform),

}

train_loader = DataLoader(
    MEUMNIST('../input/digit-recognizer/train.csv', 
             transform=data_transform),
             batch_size=40, shuffle=True, num_workers=4)

test_loader = DataLoader(
    MEUMNIST('../input/digit-recognizer/test.csv', 
             transform=data_transform),
             batch_size=40, shuffle=True, num_workers=4)


In [ ]:
image_batch, labels = next(iter(train_loader))
print(labels.shape)
print(image_batch.shape)
print(labels)

In [ ]:
grid = torchvision.utils.make_grid(image_batch, normalize=True, pad_value=1.0,padding=1)
plt.figure(figsize=(20,10))
plt.imshow(grid.numpy().transpose(1,2,0))
plt.axis("off")
plt.title("Numeros")
plt.show()

In [ ]:
dataloaders = {'train': torch.utils.data.DataLoader(datasets['train'],
                                                            batch_size=100,
                                                            shuffle=True, num_workers=4),
               'test': torch.utils.data.DataLoader(datasets['train'],
                                                            batch_size=100,
                                                            shuffle=True, num_workers=4), }

datasets_sizes = {x: len(datasets[x]) for x in ['train', 'test']} #, 'val']}

print('Tamanho do conjunto de treinamento:', datasets_sizes['train'])
print('Tamanho do conjunto de teste:', datasets_sizes['test'])


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,32,3,1)
        self.conv2 = nn.Conv2d(32,64,3,1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216,128)
        self.fc2 = nn.Linear(128,10)
        
    def forward(self, x):
        x = self.conv1(x)
        x - F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x,2)
        x = self.dropout1(x)
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim =1)
        return output
    

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = Net().to(device)

In [ ]:
losses = []
def train(model,device, train_loader, optimizer, epoch):
    model.train()
    dataset_size = len(train_loader.dataset)
    for batch_idx, (data, target) in enumerate(train_loader):
        #Preparacao
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        #Feed forward da rede
        output = model(data)
        #Calculo do erro
        loss = F.nll_loss(output, target) # Negative lof likelihood loss
        #backpropafgation
        loss.backward()
        optimizer.step()
        #print
        batch_size = len(data)
        number_of_batches = len(train_loader)
        current_loss = loss.item()
        if batch_idx % 100 ==0:
           # losses.append(loss.item())
            losses.append(loss.cpu().data.item())
            print("Train Epoch: %d [ %d/%d (%.0f%%)]\tLoss: %.6f"%(
                epoch+1,batch_idx * batch_size, dataset_size,
                100.0 * batch_idx/number_of_batches, current_loss))

In [ ]:
def test(model,device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0 
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += F.nll_loss(output,target, reduction = 'sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
          
    test_loss /= len(test_loader.dataset)

    print('test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%\n'.format(
    test_loss,correct,len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [ ]:
    optimizer = optim.Adadelta(model.parameters(), lr=0.1)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.7)

    n_epochs = 10
    for epoch in range(0, n_epochs):
        train(model, device, dataloaders['train'], optimizer, epoch)
        test(model,device,dataloaders['test'])
        scheduler.step()

In [ ]:
plt.plot(losses)


In [ ]:
temp_dataloader = torch.utils.data.DataLoader(datasets['train'],
                                             batch_size=28000,
                                             shuffle=True, num_workers=0)
image_batch, labels = next(iter(temp_dataloader))

image_batch = image_batch.to(device)
example_output = model.forward(image_batch)
_,y_hat = torch.max(example_output,1)

x = image_batch.cpu().detach().numpy()

w=2
h=2
fig=plt.figure(figsize=(12,10))
columns =2
rows = 2

for i in range(2, columns*rows +1):
    fig.add_subplot(rows, columns,i)
    plt.imshow(x[i,0].reshape(28,28),interpolation='nearest',cmap='gray')
    plt.colorbar()
    output = y_hat[i].cpu().numpy()
    print(output)
    
plt.show()


In [ ]:
y = torch.Tensor.cpu(y_hat).detach().numpy()
image_id = np.arange(1, len(y) + 1)

result = np.hstack((image_id.reshape((-1, 1)), y.reshape(-1,1)))
final_output = pd.DataFrame(result, columns=['ImageId', 'Label'])

final_output.to_csv(path_or_buf ="submission.csv", index=False)


In [ ]:
final_output